In [ ]:
from matplotlib import pyplot as plt

%load_ext autoreload
%autoreload 2
%config InlineBackend.figure_format = 'retina'

In [ ]:
GPTFILE = """
# Start divergent electron beam polarised in the x-direction
gs = 2.00231930436256 ;
GBz = 2.0 ;
pr = 0.1 ;
setparticles("beam",1000,me,qe,0,gs) ;
setGBrxydist("beam","u",pr/2,pr) ;
setGBphidist("beam","u",0,2*pi) ;
setGBzdist("beam","u",GBz,0) ;
setspinxdist("beam","u",1,0) ;

# Set-up
nI = 6600 ;
radius = 0.1 ;
solenoid("wcs","z",0.5, radius,nI) ;

# Tracking and output
accuracy(5,5,6) ;
tout(0,1.2/c,0.01/c) ;
#screen("wcs","I",1) ;

# Estimated spin-rotation
pp("Estimated spin rotation:", -(qe/(me*c*GBz)) * mu0*nI , "radians") ;

"""

In [ ]:
with open('gptspin.in', 'w') as fid:
    fid.write(GPTFILE)
    

In [ ]:
from gpt import GPT

In [ ]:
G = GPT('gptspin.in', spin_tracking=True)

In [ ]:
G.run()

In [ ]:
G.tout_stat('spin_polarization')

In [ ]:
plt.plot(G.tout_stat('mean_z'), G.tout_stat('spin_polarization'))
plt.xlabel('z (m)')
plt.ylabel('Polarization')

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
import numpy as np

#soa = np.array([[0, 0, 1, 1, -2, 0], [0, 0, 2, 1, 1, 0],
#                [0, 0, 3, 2, 1, 0], [0, 0, 4, 0.5, 0.7, 0]])

#X, Y, Z, U, V, W = zip(*soa)

X, Y, Z = G.tout_stat('mean_spinx'), G.tout_stat('mean_spiny'), G.tout_stat('mean_spinz')

f = 5

n = 2

SX, SY, SZ = G.tout_stat('mean_spinx'), G.tout_stat('mean_spiny'), G.tout_stat('mean_spinz')

fig = plt.figure(figsize=(5, 15))
ax = fig.add_subplot(111, projection='3d')
ax.quiver(Z[::n], X[::n]*1e6, Y[::n]*1e6, f*SZ[::n], f*SX[::n], f*SY[::n])
ax.plot(Z, X*1e6, Y*1e6)

ax.set_xlabel('z (m)')
ax.set_ylabel(r'x ($\mu$m)')
ax.set_zlabel(r'y ($\mu$m)')

plt.tight_layout()


In [ ]:
plt.plot(G.tout_stat('mean_z'), G.tout_stat('mean_spinx'))
plt.plot(G.tout_stat('mean_z'), G.tout_stat('mean_spiny'))
plt.plot(G.tout_stat('mean_z'), G.tout_stat('mean_spinz'))

In [ ]:
plt.plot(G.tout_stat('mean_spinx'), G.tout_stat('mean_spiny'))
plt.xlabel(r'$<S_x>$')
plt.ylabel(r'$<S_y>$')

In [ ]:
stds = np.sqrt(G.tout_stat('sigma_spinx')**2 + G.tout_stat('sigma_spiny')**2 + G.tout_stat('sigma_spinx')**2 + G.tout_stat('sigma_spinz')**2)

plt.plot(G.tout_stat('mean_z'), stds)


plt.xlabel('z (m)')
plt.ylabel(r'$\sqrt{\sigma_{s_x}^2+\sigma_{s_y}^2 + \sigma_{s_z}^2}$')

In [ ]:
import os 

os.remove('gptspin.in')